In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')
model = AutoModel.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')


def compute_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings

d:\Files\HM\holo-mentor-ml\hm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = [
    "ප්‍රත්‍යවර්තක ධාරාවක්‌ සඳහා සමීකරණය 1 ~ 1, ල $” ලෙස ලිවිය හැකිය. මෙහි ! යනු කාලය වන අතර 1",
    "ශුන්‍ය ලක්ෂ්‍ය ශක්තිය යනු ක්වොන්ටම් යාන්ත්‍රික භෞතික පද්ධතියකට තිබිය හැකි අවම ශක්තියයි. “පහත සඳහන් අය අතරින් මෙම සංකල්පය ඉදිරිපත් කළේ කවුද?",
    "Night Vision උපාංගවල බහුලව භාවිතා වන තරංග අතරින් පහත සඳහන් තරංග මොනවාද?",
    "පහත සඳහන් දේවලින් දර්පණ සූත්‍රය කුමක්ද?[A] v + u =f [B] 1/v + 1/u = 1/f [C] 1/v + 1/u = f [D] ඉහත කිසිවක් නොවේ",
    "වස්තුව අනන්තයේ තැබූ විට උත්තල දර්පණයකින් සෑදෙන රූපයේ ස්වභාවය කුමක්ද?",
    "ගැල්වනික් සෛලයක ඉලෙක්ට්‍රෝලය හරහා ධාරාව ගමන් කරන දිශාව කුමක්ද?",
    "සන්නායක සම්බන්ධයෙන් පහත සඳහන් දේවලින් නිවැරදි වන්නේ කුමක්ද?",
    "මෝටර් රථ රියදුරන් පිටුපස මාර්ගය බැලීමට භාවිතා කරන්නේ කුමන ආකාරයේ දර්පණයක්ද?",
    "සෙල්සියස් සහ ෆැරන්හයිට් පරිමාණයන් මත සංඛ්‍යාත්මක අගයන් සමාන වන්නේ කුමන උෂ්ණත්වයකදීද (සෙල්සියස් අංශකයෙන්),?",

    "What is the rate of flow of electric charges called?",
    " Which of the following is the SI Unit of Electric Current?",
    "When one unit electric charge moves from one point to another point in an electric circuit, then the amount of work done in joules is known as",
    "The hindrance presented by material of conductor to the smooth passing of electric current is known as",
    "The resistance of a conductor depends on",
    "Which of the following does not produce a sound wave?",
    "Which of the following is the unit for frequency",
    "he amplitude of a wave is the distance between two consecutive",
    "Three rays of light are incident on the boundary between a glass block and air."
]


In [3]:
# Compute embeddings
embeddings = compute_embeddings(documents)
print("Embeddings shape:", embeddings.shape)


Embeddings shape: (18, 768)


In [15]:
# Search
query = "ප්‍රත්‍යවර්තක ධාරාවක්‌"
query_embedding = compute_embeddings([query])


In [16]:

cosine_similarities = torch.nn.functional.cosine_similarity(
    torch.tensor(query_embedding),
    torch.tensor(embeddings)
)


results = list(zip(documents, cosine_similarities.tolist()))

results.sort(key=lambda x: x[1], reverse=True)

for i, (doc, score) in enumerate(results):
    print(f"{i+1:2d}. {doc} ({score:.2f})")

 1. සන්නායක සම්බන්ධයෙන් පහත සඳහන් දේවලින් නිවැරදි වන්නේ කුමක්ද? (0.94)
 2. වස්තුව අනන්තයේ තැබූ විට උත්තල දර්පණයකින් සෑදෙන රූපයේ ස්වභාවය කුමක්ද? (0.85)
 3. ප්‍රත්‍යවර්තක ධාරාවක්‌ සඳහා සමීකරණය 1 ~ 1, ල $” ලෙස ලිවිය හැකිය. මෙහි ! යනු කාලය වන අතර 1 (0.83)
 4. ගැල්වනික් සෛලයක ඉලෙක්ට්‍රෝලය හරහා ධාරාව ගමන් කරන දිශාව කුමක්ද? (0.77)
 5. ශුන්‍ය ලක්ෂ්‍ය ශක්තිය යනු ක්වොන්ටම් යාන්ත්‍රික භෞතික පද්ධතියකට තිබිය හැකි අවම ශක්තියයි. “පහත සඳහන් අය අතරින් මෙම සංකල්පය ඉදිරිපත් කළේ කවුද? (0.70)
 6. පහත සඳහන් දේවලින් දර්පණ සූත්‍රය කුමක්ද?[A] v + u =f [B] 1/v + 1/u = 1/f [C] 1/v + 1/u = f [D] ඉහත කිසිවක් නොවේ (0.68)
 7. When one unit electric charge moves from one point to another point in an electric circuit, then the amount of work done in joules is known as (0.67)
 8. Which of the following is the unit for frequency (0.65)
 9. he amplitude of a wave is the distance between two consecutive (0.64)
10. සෙල්සියස් සහ ෆැරන්හයිට් පරිමාණයන් මත සංඛ්‍යාත්මක අගයන් සමාන වන්නේ කුමන උෂ්ණත්වයකදීද (සෙල්සියස් අංශකයෙන්),? (0